In [ ]:
import heapq
from collections import deque

def input_state(prompt):
    print(prompt)
    values = list(map(int, input().split()))
    return tuple([tuple(values[i:i+3]) for i in range(0, 9, 3)])

def find_blank(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def is_goal(state, goal):
    return state == goal

def get_successors(state):
    x, y = find_blank(state)
    directions = {'Up': (-1, 0), 'Down': (1, 0), 'Left': (0, -1), 'Right': (0, 1)}
    successors = []

    for action, (dx, dy) in directions.items():
        nx, ny = x + dx, y + dy
        if 0 <= nx < 3 and 0 <= ny < 3:
            new_state = [list(row) for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]
            successors.append((tuple(tuple(row) for row in new_state), action))
    return successors

def bfs(start, goal):
    queue = deque([(start, [])])
    visited = set()
    order = []

    while queue:
        state, path = queue.popleft()
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for succ, _ in get_successors(state):
            if succ not in visited:
                queue.append((succ, path + [state]))
    return order, None

def dfs(start, goal, visited=None, path=None, order=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []
    if order is None:
        order = []

    order.append(start)
    if is_goal(start, goal):
        return order, path + [start]
    visited.add(start)

    for succ, _ in get_successors(start):
        if succ not in visited:
            result = dfs(succ, goal, visited, path + [start], order)
            if result[1]:
                return result
    return order, None

def dls(start, goal, limit, path=None, order=None):
    if path is None:
        path = []
    if order is None:
        order = []

    order.append(start)
    if is_goal(start, goal):
        return order, path + [start]
    if limit <= 0:
        return order, None

    for succ, _ in get_successors(start):
        if succ not in path:
            result = dls(succ, goal, limit - 1, path + [start], order)
            if result[1]:
                return result
    return order, None

def ids(start, goal, max_depth=20):
    total_order = []
    for depth in range(max_depth):
        order, result = dls(start, goal, depth)
        total_order.extend(order)
        if result:
            return total_order, result
    return total_order, None

def ibs(start, goal, max_breadth=20):
    visited = set()
    order = []
    queue = deque([(start, [])])

    breadth = 1
    while breadth <= max_breadth:
        next_level = deque()
        level_count = 0
        while queue and level_count < breadth:
            state, path = queue.popleft()
            order.append(state)
            if is_goal(state, goal):
                return order, path + [state]
            if state in visited:
                continue
            visited.add(state)
            for succ, _ in get_successors(state):
                if succ not in visited:
                    next_level.append((succ, path + [state]))
                    level_count += 1
        queue = next_level
        breadth += 1
    return order, None

def ucs(start, goal):
    heap = [(0, start, [])]
    visited = set()
    order = []

    while heap:
        cost, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for succ, _ in get_successors(state):
            if succ not in visited:
                heapq.heappush(heap, (cost + 1, succ, path + [state]))
    return order, None

def manhattan(state, goal):
    dist = 0
    goal_pos = {val: (i, j) for i, row in enumerate(goal) for j, val in enumerate(row)}
    for i in range(3):
        for j in range(3):
            val = state[i][j]
            if val != 0:
                gi, gj = goal_pos[val]
                dist += abs(i - gi) + abs(j - gj)
    return dist

def greedy(start, goal):
    heap = [(manhattan(start, goal), start, [])]
    visited = set()
    order = []

    while heap:
        h, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for succ, _ in get_successors(state):
            if succ not in visited:
                heapq.heappush(heap, (manhattan(succ, goal), succ, path + [state]))
    return order, None

def astar(start, goal):
    heap = [(manhattan(start, goal), 0, start, [])]
    visited = set()
    order = []

    while heap:
        f, cost, state, path = heapq.heappop(heap)
        order.append(state)
        if is_goal(state, goal):
            return order, path + [state]
        if state in visited:
            continue
        visited.add(state)
        for succ, _ in get_successors(state):
            if succ not in visited:
                new_cost = cost + 1
                heapq.heappush(heap, (new_cost + manhattan(succ, goal), new_cost, succ, path + [state]))
    return order, None

def print_state_path(path):
    for step in path:
        for row in step:
            print(row)
        print("------")

def main():
    start = input_state("Enter initial state (9 space-separated numbers):")
    goal = input_state("Enter goal state (9 space-separated numbers):")

    algorithms = {
        'BFS': bfs,
        'DFS': dfs,
        'DLS (limit=10)': lambda s, g: dls(s, g, 10),
        'IDS': ids,
        'IBS': ibs,
        'UCS': ucs,
        'Greedy Best First': greedy,
        'A*': astar
    }

    for name, func in algorithms.items():
        print(f"\n=== {name} ===")
        order, path = func(start, goal)
        print(f"Visited {len(order)} states.")
        print("Solution Path:")
        if path:
            print_state_path(path)
        else:
            print("No solution found.")

if __name__ == "__main__":
    main()


Enter initial state (9 space-separated numbers):


 1 3 5 4 2 0 7 8 6


Enter goal state (9 space-separated numbers):


 1 2 3 4 5 6 7 8 0
